In [4]:
from bs4 import BeautifulSoup as bs
import os
import os.path
import re

RAW_HANSARD_DIR = 'scraped_content'

In [5]:
for file in os.listdir(RAW_HANSARD_DIR):
    filepath = os.path.join(RAW_HANSARD_DIR, file)
    if not os.path.isfile(filepath):
        continue
    
    print(f'\nProcessing: {file}')
    
    with open(filepath, 'r') as f:
        soup = bs(f, 'html.parser')
        
    sit_date_str = soup.find('meta', {'name': 'Sit_Date'})['content']
    parl_np_str = soup.find('meta', {'name': 'Parl_No'})['content']
    
    report_table = soup.find('div', {'class': 'reportTable'})
    raw_text = report_table.get_text()
    first_number_dot_occurrence = re.search(r'\d+\. ', raw_text)
    
    if not first_number_dot_occurrence:
        print("Doesn't seem to be PQs (no number + period + space substring)")
        continue
    
    # note: can't use honorifics as part of filtering criteria as the programmer doesn't have an exhaustive list
    # of honorifics and some of the honorifics are very obscure ones (e.g. Inche in Inche Rahamat Bin Kenap)
    raw_text = raw_text[first_number_dot_occurrence.span()[0]:]
    raw_text = raw_text.replace('\n\n', ' ')
    raw_paras = raw_text.split('\xa0\xa0\xa0\xa0')
    
    # i think honorific + name should take between 8 and 75 chars. 
    pq_paras = list(
        map(lambda s: re.sub('Column: \d+', '', re.sub('\d+. ', '', s.replace('  ', ' ').strip())), 
            filter(lambda para: re.match(r'\d+\. .{8,75} asked the ', para), raw_paras)))
    
    if not pq_paras:
        print("Doesn't seem to be PQs (no one asked anything)")
        continue
        
    print(pq_paras)
    


Processing: sprs3topic_reportid=motion-1922.html


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 421685: character maps to <undefined>